In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist
# from SourceCodeTools.models.graph.train.objectives.SCAAClassifierObjetive import SCAAClassifierObjective
import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
from collections import OrderedDict
from itertools import chain

import torch

from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphClassifierObjective


class PoolingLayer(torch.nn.Module):

    def __init__(self, k, shape=1):
        super().__init__()
        self.learnable_vector = torch.nn.Parameter(torch.randn(shape))
        self.learnable_vector.requires_grad = True
        self.k = k

    def forward(self, x):
        length = torch.norm(self.learnable_vector)
        y = torch.mm(x, self.learnable_vector)/length
        idx = torch.topk(y, self.k)
        x_part = x[idx, :]
        return torch.mul(x_part, torch.transpose(y))


class SCAAClassifierObjective(SubgraphClassifierObjective):

    def __init__(
            self, name, graph_model, node_embedder, nodes, data_loading_func, device,
            sampling_neighbourhood_size, batch_size,
            tokenizer_path=None, target_emb_size=None, link_predictor_type="inner_prod",
            masker = None, measure_scores=False, dilate_scores=1,
            early_stopping=False, early_stopping_tolerance=20, nn_index="brute",
            ns_groups=None, subgraph_mapping=None, subgraph_partition=None
        ):
        SubgraphClassifierObjective.__init__(self,
            name, graph_model, node_embedder, nodes, data_loading_func, device,
            sampling_neighbourhood_size, batch_size,
            tokenizer_path, target_emb_size, link_predictor_type,
            masker, measure_scores, dilate_scores, early_stopping, early_stopping_tolerance, nn_index,
            ns_groups, subgraph_mapping, subgraph_partition
        )
        self.pooler = PoolingLayer(50, (1, 128))
        
    def parameters(self, recurse: bool = True):
        return chain(self.classifier.parameters(), self.pooler.parameters())

    def custom_state_dict(self):
        state_dict = OrderedDict()
        for k, v in self.classifier.state_dict().items():
            state_dict[f"classifier.{k}"] = v
        for k, v in self.pooler.state_dict().items():
            state_dict[f"pooler.{k}"] = v
        return state_dict

    def custom_load_state_dict(self, state_dicts):
        self.classifier.load_state_dict(
            self.get_prefix("classifier", state_dicts)
        )
        self.pooler.load_state_dict(
            self.get_prefix("pooler", state_dicts)
        )


In [4]:
tokenizer_path = "examples/sentencepiece_bpe.model"

data_path = "examples/scaa"
subgraph_partition = join(data_path, "subgraph_partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [5]:
unpersist(filecontent_path)

,id,id.1,filecontent,user,task,year,package
0,489,489,"""""""\n Author : thekushalghosh\n Team : C...",thekushalghosh,3,2020,thekushalghosh
1,474,474,T=int(input())\nimport math\nfor i in range(T)...,shubhi_,6,2020,shubhi_
2,310,310,"T,B=[int(x) for x in input().split()]\nfor x i...",briangodwinlim,4,2020,briangodwinlim
3,80,80,"from __future__ import division, print_functio...",Itachi_uchiha,8,2020,Itachi_uchiha
4,374,374,import sys\n\n\ndef main():\n cases = int(in...,erdnase,5,2020,erdnase
...,...,...,...,...,...,...,...
508,48,48,def extractLine():\n line = input()\n arr ...,DavidEdey,3,2020,DavidEdey
509,108,108,num_tests = int(input())\n\ndef sol(pairs):\n ...,KimJohnWu,0,2020,KimJohnWu
510,484,484,import sys\n\ndef printsmallpath(n): # n <= 50...,spelvin,7,2020,spelvin
511,406,406,def process_matrix(matrix):\n N = len(matrix...,luctchak,1,2020,luctchak


In [6]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,68048,49684,489,NaN
1,1,name,49684,32062,489,20349.0
2,2,name_rev,32062,49684,489,20349.0
3,3,names,32062,75398,489,20349.0
4,4,names_rev,75398,32062,489,20349.0
5,5,subword,48908,19829,489,NaN
6,6,name,19829,70699,489,20349.0
7,7,name_rev,70699,19829,489,20349.0
8,8,names,70699,75398,489,20349.0
9,9,names_rev,75398,70699,489,20349.0


In [7]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    epochs=1000,
    elem_emb_size=50,
)

In [8]:
save_config(config, "var_misuse_tiny.yaml")

In [9]:
config

{'DATASET': {'data_path': 'scaa',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': 'scaa/subgraph_partition.json.bz2'},
 'TRAINING': {'model_output_dir': 'scaa',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 50,
  'embedding_table_size': 200000,
  'epochs': 1000,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': False,
  'use_ns_groups': False,
  'nn_index': 'brute',
  'metric': 'inne

# Create Dataset

In [10]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

/home/alina/workspace/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.nodes = self.nodes.append(new_nodes, ignore_index=True)
/home/alina/workspace/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.edges = self.edges.append(new_edges, ignore_index=True)


# Declare target loading function (labels)

In [11]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "user"]].rename({"id": "src", "user": "dst"}, axis=1)

One or several objectives could be used

In [12]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SCAAClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [13]:
# %tensorboard --logdir data
from tensorboard import program

tracking_address = './scaa/' # the path of your log file.

tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', tracking_address])
url = tb.launch()
print(f"Tensorflow listening on {url}")


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



Tensorflow listening on http://localhost:6006/


In [14]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Epoch 0:   0%|          | 0/3 [00:00<?, ?it/s]

Number of nodes 80375


Epoch 0:   0%|          | 0/3 [00:10<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x100 and 50x100)